# DECISION TREE Y RANDOM FOREST
---

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')

In [2]:
df_est = pd.read_csv("datos/05-bikes_encoding_est.csv", index_col = 0)
df_est.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,registered
0,0,0,0,0,0,1,2,14.110847,-0.680818,1.252343,-0.387833,654
1,0,0,0,7,2,2,2,14.902598,-0.741507,0.480996,0.748899,670
2,0,0,1,7,2,2,4,8.050924,-1.750344,-1.338073,0.745931,1229
3,1,0,1,7,1,1,4,8.200000,-1.610886,-0.261577,-0.389769,1454
4,1,0,2,7,1,2,4,9.305237,-1.505615,-1.340294,-0.046477,1518


In [3]:
df_est.season.unique()

array([0, 1, 2], dtype=int64)

In [4]:
df_est.isnull().sum()

season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
registered    0
dtype: int64

In [5]:
# Creamos una clase para poder hacer el decision tree y el random forest más rápido
class Regres_lineal:
    def __init__(self, dataf, col_pred):
        
        self.dataf = dataf
        self.col_pred = col_pred

    def tree_param(self):
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        arbol = DecisionTreeRegressor(random_state =0)
    
        arbol.fit(x_train, y_train)

        max_feat = np.sqrt(len(x_train.columns))
        max_dep = arbol.tree_.max_depth
        return(max_feat, max_dep)

    def metricas(self, y_te, y_tr, y_te_pred, y_tr_pred, tipo_modelo):
        resultados = {'MAE': [mean_absolute_error(y_te, y_te_pred), mean_absolute_error(y_tr, y_tr_pred)],
                    'MSE': [mean_squared_error(y_te, y_te_pred), mean_squared_error(y_tr, y_tr_pred)],
                    'RMSE': [np.sqrt(mean_squared_error(y_te, y_te_pred)), np.sqrt(mean_squared_error(y_tr, y_tr_pred))],
                    'R2':  [r2_score(y_te, y_te_pred), r2_score(y_tr, y_tr_pred)],
                    "set": ["test", "train"]}
        df = pd.DataFrame(resultados)
        df["modelo"] = tipo_modelo
        return df

    def grid_s_decision(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= GradientBoostingRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

    def grid_s_forest(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= RandomForestRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

In [6]:

rg_est = Regres_lineal(df_est, "registered")

In [7]:
features_est, depth_est = rg_est.tree_param()

In [8]:
features_est #El número máx será 3

3.3166247903554

In [9]:
depth_est # La profundidad máxima será 20

20

In [10]:
param1 = {"max_depth": [16, 18, 20], 
        "max_features": [3,4],
        "min_samples_split": [50, 100],
        "min_samples_leaf": [50,100]} 

In [11]:
best_model1, result1 = rg_est.grid_s_decision(param1, "GradientBoosting Estand I")

In [12]:
best_model1

GradientBoostingRegressor(max_depth=18, max_features=3, min_samples_leaf=50,
                          min_samples_split=50)

In [13]:
result1

,MAE,MSE,RMSE,R2,set,modelo
0,520.571870,434750.762621,659.356324,0.798593,test,GradientBoosting Estand I
1,479.401543,381938.370705,618.011627,0.846801,train,GradientBoosting Estand I


In [14]:
param2 = {"max_depth": [4,6,8,10], 
        "max_features": [2, 3,4],
        "min_samples_split": [ 40, 50, 60],
        "min_samples_leaf": [10, 20]} 

In [15]:
best_model2, result2 = rg_est.grid_s_decision(param2, "GradientBoosting Estand II")

In [16]:
best_model2

GradientBoostingRegressor(max_depth=4, max_features=2, min_samples_leaf=10,
                          min_samples_split=60)

In [17]:
result2

,MAE,MSE,RMSE,R2,set,modelo
0,569.474169,489623.668009,699.731140,0.773172,test,GradientBoosting Estand II
1,434.667915,324736.291989,569.856378,0.869745,train,GradientBoosting Estand II


In [18]:
param3 = {"max_depth": [ 12, 14, 16, 18], 
        "max_features": [3,4],
        "min_samples_split": [10, 20, 40],
        "min_samples_leaf": [10, 20, 40]} 

In [19]:
best_model3, result3 = rg_est.grid_s_forest(param3, "RandForest Estand I")

In [20]:
best_model3

RandomForestRegressor(max_depth=16, max_features=4, min_samples_leaf=10,
                      min_samples_split=20)

In [21]:
result3

,MAE,MSE,RMSE,R2,set,modelo
0,579.153264,540720.153101,735.336762,0.749501,test,RandForest Estand I
1,562.571349,517729.338741,719.534112,0.792334,train,RandForest Estand I


In [22]:
param4 = {"max_depth": [12, 14], 
        "max_features": [4,5],
        "min_samples_split": [ 40, 50],
        "min_samples_leaf": [10, 20,]} 

In [23]:
best_model4, result4 = rg_est.grid_s_forest(param4, "RandForest Estand II")

In [24]:
best_model4

RandomForestRegressor(max_depth=14, max_features=5, min_samples_leaf=10,
                      min_samples_split=40)

In [25]:
result4

,MAE,MSE,RMSE,R2,set,modelo
0,595.95045,564242.198557,751.160568,0.738604,test,RandForest Estand II
1,592.09850,571558.619775,756.014960,0.770742,train,RandForest Estand II


In [26]:
df_sinest = pd.read_csv("datos/04-bikes_encoding_sinest.csv", index_col = 0)
df_sinest.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,registered
0,0,0,0,0,0,1,2,14.110847,18.18125,80.5833,10.749882,654
1,0,0,0,7,2,2,2,14.902598,17.68695,69.6087,16.652113,670
2,0,0,1,7,2,2,4,8.050924,9.47025,43.7273,16.636703,1229
3,1,0,1,7,1,1,4,8.200000,10.60610,59.0435,10.739832,1454
4,1,0,2,7,1,2,4,9.305237,11.46350,43.6957,12.522300,1518


In [27]:
rg_sinest = Regres_lineal(df_sinest, "registered")

In [28]:
features_sinest, depth_sinest = rg_sinest.tree_param()

In [29]:
features_sinest #El número máx será 3

3.3166247903554

In [30]:
depth_sinest # La profundidad máxima será 22

20

In [31]:
best_model5, result5 = rg_sinest.grid_s_decision(param1, "GradientBoosting SinEstand I")

In [32]:
result5

,MAE,MSE,RMSE,R2,set,modelo
0,512.455584,424609.776623,651.620884,0.803291,test,GradientBoosting SinEstand I
1,477.662737,385606.127572,620.971922,0.845330,train,GradientBoosting SinEstand I


In [33]:
best_model6, result6 = rg_sinest.grid_s_decision(param2, "GradientBoosting SinEstand II")

In [34]:
result6

,MAE,MSE,RMSE,R2,set,modelo
0,564.079377,490123.682414,700.088339,0.772941,test,GradientBoosting SinEstand II
1,348.698544,208994.939048,457.159643,0.916170,train,GradientBoosting SinEstand II


In [35]:
best_model7, result7 = rg_sinest.grid_s_forest(param3, "RandForest SinEstand I")

In [36]:
result7

,MAE,MSE,RMSE,R2,set,modelo
0,575.046389,546939.849880,739.553818,0.746620,test,RandForest SinEstand I
1,559.659781,515747.640631,718.155722,0.793128,train,RandForest SinEstand I


In [37]:
best_model8, result8 = rg_sinest.grid_s_forest(param4, "RandForest SinEstand II")

In [38]:
result8

,MAE,MSE,RMSE,R2,set,modelo
0,591.322904,556470.444967,745.969467,0.742204,test,RandForest SinEstand II
1,593.247550,569062.350919,754.362215,0.771743,train,RandForest SinEstand II


In [39]:
param5 = {"max_depth": [22], 
        "max_features": [4],
        "min_samples_split": [8],
        "min_samples_leaf": [8]}

In [40]:
best_model9, result9 = rg_sinest.grid_s_forest(param5, "RandForest SinEstand III")

In [41]:
result9

,MAE,MSE,RMSE,R2,set,modelo
0,565.929324,529612.970218,727.745127,0.754647,test,RandForest SinEstand III
1,533.135543,476052.540364,689.965608,0.809051,train,RandForest SinEstand III


In [42]:
best_model10, result10 = rg_est.grid_s_forest(param5, "RandForest Estand III")

In [43]:
result10

,MAE,MSE,RMSE,R2,set,modelo
0,580.240894,546932.369773,739.548761,0.746623,test,RandForest Estand III
1,533.817496,475444.879013,689.525111,0.809294,train,RandForest Estand III


In [44]:
param6 = {"max_depth": [18,20], 
        "max_features": [3,4],
        "min_samples_split": [8, 15],
        "min_samples_leaf": [8, 15]}

In [45]:
best_model11, result11 = rg_est.grid_s_forest(param6, "RandForest Estand IV")

In [46]:
best_model11

RandomForestRegressor(max_depth=20, max_features=4, min_samples_leaf=8,
                      min_samples_split=15)

In [47]:
result11

,MAE,MSE,RMSE,R2,set,modelo
0,560.842914,512722.628673,716.046527,0.762471,test,RandForest Estand IV
1,536.981049,473381.174208,688.027016,0.810122,train,RandForest Estand IV


In [48]:
best_model12, result12 = rg_est.grid_s_decision(param5, "GradientBoosting Estand III")

In [49]:
result12

,MAE,MSE,RMSE,R2,set,modelo
0,562.415355,512931.202522,716.192155,0.762375,test,GradientBoosting Estand III
1,94.681003,18861.146486,137.335889,0.992435,train,GradientBoosting Estand III


In [50]:
best_model13, result13 = rg_sinest.grid_s_decision(param5, "GradientBoosting SinEstand III")

In [51]:
best_model13

GradientBoostingRegressor(max_depth=22, max_features=4, min_samples_leaf=8,
                          min_samples_split=8)

In [52]:
result13

,MAE,MSE,RMSE,R2,set,modelo
0,575.042180,536013.930495,732.129722,0.751681,test,GradientBoosting SinEstand III
1,95.932904,19972.387827,141.323699,0.991989,train,GradientBoosting SinEstand III


In [53]:
param7 = {"max_depth": [18,20], 
        "max_features": [4],
        "min_samples_split": [8, 16],
        "min_samples_leaf": [8]}

In [54]:
best_model14, result14 = rg_est.grid_s_decision(param7, "GradientBoosting Estand IV")

In [55]:
best_model14

GradientBoostingRegressor(max_depth=20, max_features=4, min_samples_leaf=8,
                          min_samples_split=16)

In [56]:
result14

,MAE,MSE,RMSE,R2,set,modelo
0,558.942177,498812.910691,706.266883,0.768915,test,GradientBoosting Estand IV
1,95.547588,19352.103075,139.111837,0.992238,train,GradientBoosting Estand IV


In [57]:
param17 = {"max_depth": [10], 
        "max_features": [3],
        "min_samples_split": [50],
        "min_samples_leaf": [10]}

best_model15, result15 = rg_est.grid_s_decision(param17, "GradientBoosting Estand IV")

result15

,MAE,MSE,RMSE,R2,set,modelo
0,546.571622,460705.534461,678.752926,0.786569,test,GradientBoosting Estand IV
1,269.308292,128200.798817,358.051391,0.948577,train,GradientBoosting Estand IV


In [58]:
param17 = {"max_depth": [16], 
        "max_features": [4],
        "min_samples_split": [40],
        "min_samples_leaf": [20]}

best_model16, result16 = rg_sinest.grid_s_decision(param17, "GradientBoosting SinEstand IV")

result16

,MAE,MSE,RMSE,R2,set,modelo
0,557.971922,473707.674004,688.264247,0.780546,test,GradientBoosting SinEstand IV
1,296.350209,158255.588369,397.813510,0.936522,train,GradientBoosting SinEstand IV
